## Store Voters in Sqlite database
We will store users in sqlite database, getting data from an excel spreadsheet

In [7]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook

class Voter(object):
    def __init__(self, firstname, lastname, idnumber, ethnicity, province_address,
                 district_address, national_vote, provincial_vote, district_vote):
        self.Firstname = firstname
        self.Lastname = lastname
        self.Idnumber = idnumber
        self.Ethnicity = ethnicity
        self.Province_address = province_address
        self.District_address = district_address
        self.National_vote = national_vote
        self.Provincial_vote = provincial_vote
        self.District_vote = district_vote

    @classmethod
    def from_excel_row(cls, row):
        """
            Creates a Voter object from a row of data in Excel.
            Returns the voter object
        """
        firstname = row[0]
        lastname = row[1]
        idnumber = row[2]
        ethnicity = row[3]
        province_address = row[4]
        district_address = row[5]
        national_vote = row[6]
        provincial_vote = row[7]
        district_vote = row[8]
        return cls(firstname, lastname, idnumber, ethnicity, province_address,
                   district_address, national_vote, provincial_vote, district_vote)

    @classmethod
    def load_from_excel(cls, file_path):
        """
        Loads voter data from an Excel file.

        Parameters:
        - file_path (str): Path to the Excel file.
        """
        voters = []
        wb = load_workbook(filename= file_path)
        ws = wb.active

        for row in ws.iter_rows(min_row=1, values_only=True):
            voter = cls.from_excel_row(row)
            voters.append(voter)

        return voters
################################# end of class

# Example usage to load data from Excel file
file_path = '../userGeneration/sample_data.xlsx'
voters = Voter.load_from_excel(file_path)

# print voter data:
for voter in voters[:2]: ## let it only print the top 2 items
    print(f"Firstname: {voter.Firstname}")
    print(f"Lastname: {voter.Lastname}")
    print(f"ID Number: {voter.Idnumber}")
    print(f"Ethnicity: {voter.Ethnicity}")
    print(f"Province: {voter.Province_address}")
    print(f"District: {voter.District_address}")
    print(f"National Vote: {voter.National_vote}")
    print(f"Provincial Vote: {voter.Provincial_vote}")
    print(f"District Vote: {voter.District_vote}")
    print("=" * 30)  



Firstname: Patricia
Lastname: Tate
ID Number: 4303147631811
Ethnicity: white
Province: Western Cape
District: Overberg
National Vote: GOOD
Provincial Vote: EFF
District Vote: ANC
Firstname: Raymond
Lastname: Pitts
ID Number: 4808312742413
Ethnicity: white
Province: Western Cape
District: Garden Route
National Vote: ALJAMA AH
Provincial Vote: ANC
District Vote: VFF


### Storing the voters in the SQLITE database "mydatabase.sqlite"

In [14]:
import sqlite3

# Loop through the voter array and add users to database:

def getEthnicityKeyGivenText(ethnicity):
    """Given the text, return the ethnicity number"""
    match ethnicity:
        case "black":
            return 1
        case "white":
            return 2
        case "coloured":
            return 3
        case "indian":
            return 4
        case _:
            return None

key = getEthnicityKeyGivenText("black")

# Function to create a connection to the database:
def create_connection(db_file = "../database/mydatabase.sqlite"):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return conn

# Get Ethnicity ID:
def get_ethnicity_id(conn, ethnicity_name):
    """Retrieve ethnicity from table, given the ethnicity string """
    cur = conn.cursor()
    cur.execute("SELECT id FROM Ethnicity WHERE name=?", (ethnicity_name,))
    row = cur.fetchone()
    if row:
        return row[0]
    else:
        return None
    
# Get or create the address:
def get_or_create_address_id(conn, province, district):
    """
    Retrieve the address ID from the Address table based on province and district.
    If the address does not exist, create a new record and return the new ID.
    """
    cur = conn.cursor()
    cur.execute("SELECT id FROM Address WHERE province=? AND district=?", (province, district))
    row = cur.fetchone()
    if row:
        return row[0]
    else:
        cur.execute("INSERT INTO Address (province, district) VALUES (?, ?)", (province, district))
        conn.commit()
        return cur.lastrowid

def get_vote_id(conn, national, provincial, regional):
    cur = conn.cursor()
    cur.execute("SELECT id FROM Vote WHERE national=? AND provincial=? AND regional=?", (national, provincial, regional))
    row = cur.fetchone()
    if row:
        return row[0]
    else:
        cur.execute("INSERT INTO Vote (national, provincial, regional) VALUES (?, ?, ?)", (national, provincial, regional))
        conn.commit()
        return cur.lastrowid


def insert_voter(conn, voter):
    """ Insert a voter into the Voter table. """
    ethnicity_id = get_ethnicity_id(conn, voter.Ethnicity)
    address_id = get_or_create_address_id(conn, voter.Province_address, voter.District_address)
    vote_id = get_vote_id(conn, voter.National_vote, voter.Provincial_vote, voter.District_vote)

    sql = ''' INSERT INTO Voter(firstname, lastname, idnumber, ethnicity_id, address_id, vote_id)
              VALUES(?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, (voter.Firstname, voter.Lastname, voter.Idnumber, ethnicity_id, address_id, vote_id))
    conn.commit()
    return cur.lastrowid



Execute the code

In [30]:
# TEST CODE
connection = create_connection()
ethnicityId = get_ethnicity_id(connection, "indian")
addressId = get_or_create_address_id(connection, "Limpopo", "Vhembe")
print(addressId)
print(ethnicityId)

1
4


In [15]:
file_path = '../userGeneration/my_data.xlsx'
voters = Voter.load_from_excel(file_path)
connection = create_connection()
with connection:
        for voter in voters:  # print only first 2 voters
            # Insert voter into database
            voter_id = insert_voter(connection, voter)